#cambio de pesos

In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ====================== Funciones Auxiliares ======================

def maape(y_true, y_pred):
    with np.errstate(divide='ignore', invalid='ignore'):
        arctan = np.arctan(np.abs((y_true - y_pred) / (y_true + 1e-10)))
    return np.mean(arctan)

def validate_input(df, required_columns, df_name):
    if not isinstance(df, pd.DataFrame):
        raise ValueError(f"{df_name} debe ser un DataFrame de pandas.")
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"{df_name} falta columnas requeridas: {missing_cols}")

def build_series(df, year, month, window=6):
    df_sorted = df.sort_values(['year', 'month']).reset_index(drop=True)
    series = []
    current_year, current_month = year, month

    for _ in range(window):
        match = df_sorted[(df_sorted['year'] == current_year) & (df_sorted['month'] == current_month)]
        if match.empty:
            return None
        series.append(match.iloc[0])

        if current_month == 1:
            current_month = 12
            current_year -= 1
        else:
            current_month -= 1

    return pd.DataFrame(series[::-1]).reset_index(drop=True)

def build_past_series(df, year, month, window=6):
    df_sorted = df.sort_values(['year', 'month']).reset_index(drop=True)
    years = df_sorted['year'].unique()
    past_years = years[years < year]
    series_list = []

    for past_year in past_years:
        series = build_series(df_sorted, past_year, month, window)
        if series is not None:
            series_list.append((past_year, series))

    return series_list

def compare_series(base_series, past_series_list, var, base_name=None):
    results = []
    y_true = base_series[var].values

    for year, past_series in past_series_list:
        y_pred = past_series[var].values

        if len(y_true) != len(y_pred):
            warnings.warn(f"El tamaño de la serie base y {year} no coincide. Se omite este año.")
            continue

        if not all(base_series['month'].values == past_series['month'].values):
            warnings.warn(f"Los meses de la serie base y {year} no coinciden exactamente. Se omite este año.")
            continue

        results.append({
            'base': base_name if base_name else var,
            'año_comparado': year,
            'MAAPE': maape(y_true, y_pred),
            'RMSE': mean_squared_error(y_true, y_pred, squared=False),
            'MAE': mean_absolute_error(y_true, y_pred)
        })

    return results

def comparar_dataframes(prec_df, atl_df, pac_df, year, month):
    validate_input(prec_df, ['year', 'month', 'prec'], 'prec_df')
    validate_input(atl_df, ['year', 'month', 'oni'], 'atl_df')
    validate_input(pac_df, ['year', 'month', 'oni'], 'pac_df')

    prec_base = build_series(prec_df, year, month)
    atl_base = build_series(atl_df, year, month)
    pac_base = build_series(pac_df, year, month)
    #print(pac_base)

    if any(x is None for x in (prec_base, atl_base, pac_base)):
        raise ValueError(f"No se pudo construir la serie base completa para {year}-{month}.")

    prec_past = build_past_series(prec_df, year, month)
    atl_past = build_past_series(atl_df, year, month)
    pac_past = build_past_series(pac_df, year, month)
    #print(pac_past[5])

    prec_results = compare_series(prec_base, prec_past, 'prec', base_name='prec')
    atl_results = compare_series(atl_base, atl_past, 'oni', base_name='atl')
    pac_results = compare_series(pac_base, pac_past, 'oni', base_name='pac')
    #print(pac_results[5])

    all_results = prec_results + atl_results + pac_results
    #print(all_results)
    return pd.DataFrame(all_results)

def ponderar_resultados(resultados_df, pesos):
    combinados = []
    años = resultados_df['año_comparado'].unique()

    for año in sorted(años):
        datos_año = resultados_df[resultados_df['año_comparado'] == año]
        for metrica in ['MAAPE', 'RMSE', 'MAE']:
            valor = sum(fila[metrica] * pesos.get(fila['base'], 0) for _, fila in datos_año.iterrows())
            combinados.append({
                'año_comparado': año,
                'metrica': metrica,
                'valor_ponderado': valor
            })
    
    return pd.DataFrame(combinados)

def flujo_completo(prec, atl, pac, real, year, month, metrica='MAAPE', pesos=None, escenario='default'):
    if metrica not in ['RMSE', 'MAE', 'MAAPE']:
        raise ValueError("La métrica debe ser 'RMSE', 'MAE' o 'MAAPE'.")

    if pesos is None:
        pesos = {'prec': 0.5, 'pac': 0.3, 'atl': 0.2}

    resultados = comparar_dataframes(prec, atl, pac, year, month)
    ponderados = ponderar_resultados(resultados, pesos)

    top_metric = (
        ponderados[ponderados.metrica == metrica]
        .sort_values('valor_ponderado')
        .head(5)
    )

    mape = top_metric.merge(real, how='inner', left_on='año_comparado', right_on='year')

    print(f"\n Escenario: {escenario} | Año base: {year} | Mes: {month} | Métrica: {metrica}")
    print(mape[['año_comparado', 'valor_ponderado', 'sem_iell']])

    # Promedio simple
    promedio_simple = mape['sem_iell'].mean()

    # Promedio ponderado con pesos específicos
    pesos_personalizados = np.array([0.5, 0.25, 0.1, 0.075, 0.075])
    promedio_ponderado = np.average(mape['sem_iell'].values, weights=pesos_personalizados)

    print(f" Promedio simple sem_iell: {promedio_simple:.3f}")
    print(f" Promedio ponderado sem_iell: {promedio_ponderado:.3f}")

    return mape

# ====================== Carga de datos ======================

prec = pd.read_excel(r"D:\OneDrive - CGIAR\agrilac\icc\data\cengicana_prec.xlsx")

pac = pd.read_excel("D:\\OneDrive - CGIAR\\agrilac\\icc\\data\\oni_pacifico.xlsx")
atl = pd.read_excel("D:\\OneDrive - CGIAR\\agrilac\\icc\\data\\oni_atlantico.xlsx")
real = pd.read_excel("D:\\OneDrive - CGIAR\\agrilac\\icc\\data\\start_prec_cengicana.xlsx")

# ====================== Escenarios de pesos ======================

escenarios = {
    'prec_dom': {'prec': 0.7, 'pac': 0.15, 'atl': 0.15},
    'enso_dom': {'prec': 0.3, 'pac': 0.5, 'atl': 0.2},
    'enso_prec': {'prec': 0.4, 'pac': 0.4, 'atl': 0.2},
    'neutro': {'prec': 0.33, 'pac': 0.33, 'atl': 0.33},
    'atl_dom': {'prec': 0.3, 'pac': 0.2, 'atl': 0.5},
}



In [2]:
year = 2007
metrica = 'MAAPE'

In [3]:
escenario = 'enso_dom'
pesos = escenarios[escenario]

print(f"\n======== RESULTADOS PARA ESCENARIO: {escenario.upper()} ========")
mape_month4 = flujo_completo(prec, atl, pac, real, year, 4, metrica=metrica, pesos=pesos, escenario=escenario)
mape_month3 = flujo_completo(prec, atl, pac, real, year, 3, metrica=metrica, pesos=pesos, escenario=escenario)


======== RESULTADOS PARA ESCENARIO: ENSO_DOM ========

 Escenario: enso_dom | Año base: 2007 | Mes: 4 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.642579      24.0
1           2005         0.737482      22.0
2           2004         0.741796      24.0
3           2002         0.863993      23.0
4           2001         0.891884      25.0
 Promedio simple sem_iell: 23.600
 Promedio ponderado sem_iell: 23.500

 Escenario: enso_dom | Año base: 2007 | Mes: 3 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.584599      24.0
1           2005         0.624102      22.0
2           2004         0.661441      24.0
3           2002         0.809920      23.0
4           2006         0.983808      19.0
 Promedio simple sem_iell: 22.400
 Promedio ponderado sem_iell: 23.050


In [4]:
escenario = 'prec_dom'
pesos = escenarios[escenario]

print(f"\n======== RESULTADOS PARA ESCENARIO: {escenario.upper()} ========")
mape_month4 = flujo_completo(prec, atl, pac, real, year, 4, metrica=metrica, pesos=pesos, escenario=escenario)
mape_month3 = flujo_completo(prec, atl, pac, real, year, 3, metrica=metrica, pesos=pesos, escenario=escenario)


======== RESULTADOS PARA ESCENARIO: PREC_DOM ========

 Escenario: prec_dom | Año base: 2007 | Mes: 4 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.516293      24.0
1           2005         0.731403      22.0
2           2004         0.777438      24.0
3           2002         0.778432      23.0
4           2001         0.823373      25.0
 Promedio simple sem_iell: 23.600
 Promedio ponderado sem_iell: 23.500

 Escenario: prec_dom | Año base: 2007 | Mes: 3 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.493623      24.0
1           2005         0.683074      22.0
2           2002         0.702872      23.0
3           2004         0.721956      24.0
4           2000         0.839446      26.0
 Promedio simple sem_iell: 23.800
 Promedio ponderado sem_iell: 23.550


In [5]:
escenario = 'enso_prec'
pesos = escenarios[escenario]

print(f"\n======== RESULTADOS PARA ESCENARIO: {escenario.upper()} ========")
mape_month4 = flujo_completo(prec, atl, pac, real, year, 4, metrica=metrica, pesos=pesos, escenario=escenario)
mape_month3 = flujo_completo(prec, atl, pac, real, year, 3, metrica=metrica, pesos=pesos, escenario=escenario)


======== RESULTADOS PARA ESCENARIO: ENSO_PREC ========



 Escenario: enso_prec | Año base: 2007 | Mes: 4 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.613136      24.0
1           2005         0.737950      22.0
2           2004         0.751831      24.0
3           2002         0.842072      23.0
4           2001         0.873827      25.0
 Promedio simple sem_iell: 23.600
 Promedio ponderado sem_iell: 23.500

 Escenario: enso_prec | Año base: 2007 | Mes: 3 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.562360      24.0
1           2005         0.641434      22.0
2           2004         0.676622      24.0
3           2002         0.781405      23.0
4           2006         0.946666      19.0
 Promedio simple sem_iell: 22.400
 Promedio ponderado sem_iell: 23.050


In [6]:

escenario = 'neutro'
pesos = escenarios[escenario]

print(f"\n======== RESULTADOS PARA ESCENARIO: {escenario.upper()} ========")
mape_month4 = flujo_completo(prec, atl, pac, real, year, 4, metrica=metrica, pesos=pesos, escenario=escenario)
mape_month3 = flujo_completo(prec, atl, pac, real, year, 3, metrica=metrica, pesos=pesos, escenario=escenario)


======== RESULTADOS PARA ESCENARIO: NEUTRO ========

 Escenario: neutro | Año base: 2007 | Mes: 4 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.648911      24.0
1           2004         0.749568      24.0
2           2005         0.751249      22.0
3           2002         0.842516      23.0
4           2001         0.867197      25.0
 Promedio simple sem_iell: 23.600
 Promedio ponderado sem_iell: 23.800

 Escenario: neutro | Año base: 2007 | Mes: 3 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.576750      24.0
1           2005         0.650917      22.0
2           2004         0.660388      24.0
3           2002         0.773898      23.0
4           2006         0.918807      19.0
 Promedio simple sem_iell: 22.400
 Promedio ponderado sem_iell: 23.050


In [7]:
escenario = 'atl_dom'
pesos = escenarios[escenario]

print(f"\n======== RESULTADOS PARA ESCENARIO: {escenario.upper()} ========")
mape_month4 = flujo_completo(prec, atl, pac, real, year, 4, metrica=metrica, pesos=pesos, escenario=escenario)
mape_month3 = flujo_completo(prec, atl, pac, real, year, 3, metrica=metrica, pesos=pesos, escenario=escenario)


======== RESULTADOS PARA ESCENARIO: ATL_DOM ========

 Escenario: atl_dom | Año base: 2007 | Mes: 4 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.693656      24.0
1           2004         0.768791      24.0
2           2005         0.785180      22.0
3           2002         0.851258      23.0
4           2001         0.869590      25.0
 Promedio simple sem_iell: 23.600
 Promedio ponderado sem_iell: 23.800

 Escenario: atl_dom | Año base: 2007 | Mes: 3 | Métrica: MAAPE
   año_comparado  valor_ponderado  sem_iell
0           2003         0.596725      24.0
1           2004         0.662694      24.0
2           2005         0.686229      22.0
3           2002         0.767844      23.0
4           2000         0.850647      26.0
 Promedio simple sem_iell: 23.800
 Promedio ponderado sem_iell: 23.875
